<a href="https://colab.research.google.com/github/Shuwei-zhang/PowerBook/blob/main/CP_Draworld_PowerMix_OptimalRE_BalanceAREAs_2023_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **The Optimal Power Mix with the constraint of Deep Carbon Mitigation: a demo for China with one node per balance area (province generally)**

Developed by @*DRAWORLD*, July-Dec 2023, contact@draworld.org

Methodology frame，only for the illustration purpose


The following project development will be based on this, but more complicated in term of the design of the Network.

For the purpose of illustration of better coding, number and formual are NEVER hard-coded.
For better understanding of the simpler model "blackbox", see a code+number mixed demo only for Zhejiang Province here:
https://colab.research.google.com/drive/1-QzwK1mPQ8Gizzlg0lpeid1AcV5HDXz3


Updates log:
1. TBA


In [ ]:
# Install the Python Package needed for optimal power mix modeling. Scope: All Provinces, China solely.

#! pip install git+https://github.com/snakemake/snakemake #using only for large amount of scenarios-building
! pip install pandas
! pip install geopandas
! pip install matplotlib
! pip install pypsa # power system model protype
! pip install atlite
! pip install seaborn
! pip install xarray
! pip install shapely
! apt-get install -y -qq coinor-cbc # open source solver

In [ ]:
# The input and output folder in Google Drive

# input: /content/drive/MyDrive/Draworld_Cloud/WRI_Draworld/StylizedModel/Inputs/
# source:
#   Solar PV, https://www.renewables.ninja/, with Lat. 29.0000 and Lon. 120.0000 (Center of Zhejiang as representative), year 2019
#   Wind, https://www.renewables.ninja/, with Lat 29.0000 and Lon 120.0000, year 2019, Vestas V90 2000
#   Demand, https://www.ndrc.gov.cn/xxgk/zcfb/tz/201912/P020191230336066090861.pdf, year 2019

# output: /content/drive/MyDrive/Draworld_Cloud/WRI_Draworld/StylizedModel/Outputs/

from google.colab import drive
drive.mount('/content/drive')

#the unit conversion, to be cleaned.
#EUR2RMB = 7.5
#USD2RMB = 6.8

#The Chinese graph settings
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

font_path = "/content/drive/MyDrive/Notebooks/fonts/STHeitiTC-Medium-01.ttf"
CHN_prop = FontProperties(fname=font_path)

# downloading the input files from Google Drive, the profile of demand and variable renewable
!gdown 1zM8rLjlD_d8F_g2azj5em79ebNdVr6mb #solar
!gdown 1-OBbrb2wOXAcsU6IaBDWWorg2fN-bNSu #wind
!gdown 1PDt9a4FW-a9mIopjq4A1jmRB8tlagynA #demand

import pandas as pd
pv  = pd.read_csv('ninja_pv_29.0000_120.0000_BAs.csv')
win = pd.read_csv('ninja_wind_29.0000_120.0000_BAs.csv')
dem = pd.read_csv('load_NDRC_BAs_China_Draworld.csv')

print(dem.head())
#print(pv.head())
#print(win['Zhejiang'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Downloading...
From: https://drive.google.com/uc?id=1zM8rLjlD_d8F_g2azj5em79ebNdVr6mb
To: /content/ninja_wind_29.0000_120.0000_BAs.csv
100% 628k/628k [00:00<00:00, 100MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-OBbrb2wOXAcsU6IaBDWWorg2fN-bNSu
To: /content/ninja_pv_29.0000_120.0000_BAs.csv
100% 612k/612k [00:00<00:00, 165MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PDt9a4FW-a9mIopjq4A1jmRB8tlagynA
To: /content/load_NDRC_BAs_China_Draworld.csv
100% 619k/619k [00:00<00:00, 157MB/s]


In [ ]:
work_path = "/content/drive/MyDrive/Draworld_Cloud/WRI-Draworld-StylizedModel/Inputs/"

# input the excel-based other parameter setttings
ntcs = pd.read_excel(work_path + "data.xlsx",
                     sheet_name="NTC",
                     index_col=0,
                     skiprows=1,
                     usecols=list(range(1,44)),
                     dtype=str)

costs = pd.read_excel(work_path + "data.xlsx",
                     sheet_name="cost",
                     index_col=0,
                     skiprows=1,
                     usecols=list(range(0,9)),
                     dtype=str
                     )
                     #skipfooter=8)

costs = costs.fillna(0.)

print(costs)

              Technology type fixed cost variable cost capacity_extendable  \
onshore wind       generation        480            20                True   
offshore wind      generation        720            20                True   
solarPV            generation        420            20                True   
hydro              generation        640            20                True   
nuclear            generation       1010           120                True   
Gas Cycle          generation        283   610.2605863                True   
Gas Turbine        generation        260   760.3257329                True   
coal               generation        380   337.1428571                True   
Shed               generation          0         10000                True   
PumpHydro             storage        485             0                True   
Battery               storage        700             0                True   
H2                    storage       1480             0          

In [ ]:
import pypsa, logging
pypsa.pf.logger.setLevel(logging.WARNING)

selected_year = 2019 # Only 2019 data available at present, by Draworld (April,2023).

# Select demand, solar (PV) and wind profiles
selected_dem = dem.loc[(pd.DatetimeIndex(dem['datetime']).year == selected_year)]
selected_pv  = pv.loc[(pd.DatetimeIndex(pv['datetime']).year == selected_year) ]
selected_win = win.loc[(pd.DatetimeIndex(win['datetime']).year == selected_year)]

#dataflow.index. "Time step" or snapshots. Noting that "" meaning "now"， By Shuwei ZHANG, March.12, 2023
n = pypsa.Network(snapshots = selected_dem.index,name = "Draworld-Network-Annual Report-Demo")

In [ ]:
# set the energy carriers, only coal, gas, oil or as same to the Generation/Storage technology..
# check n.madd in EuR model, 22 June 2023..

